In [5]:
import numpy as np
import pandas as pd
import os
import turicreate as tc
import random
import math

from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [6]:
X = tc.SFrame("./train_X_rf.csv")
Y = tc.SFrame.read_csv("./train_Y_rf.csv", header=False)

trainX = pd.read_csv("./train_X_rf.csv")
trainY = pd.read_csv("./train_Y_rf.csv", header=None)

Finished parsing file /mnt/e/Spyder - Copy/ML Project/random_forest/train_X_rf.csv

Parsing completed. Parsed 100 lines in 0.316555 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /mnt/e/Spyder - Copy/ML Project/random_forest/train_X_rf.csv

Parsing completed. Parsed 156 lines in 0.030298 secs.

...

------------------------------------------------------
Inferred types from first 100 line(s) of file as 


Parsing completed. Parsed 100 lines in 0.029775 secs.

column_type_hints=[int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /mnt/e/Spyder - Copy/ML Project/random_forest/train_Y_rf.csv

Parsing completed. Parsed 156 lines in 0.034066 secs.

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(trainX, trainY, test_size=0.2, random_state = 0)

In [8]:
import numpy as np
import pandas as pd
import math
import pickle

from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

def gini(S):
    c = Counter(S)
    m = len(S)
    t = 0
    for i in c:
        t += (c[i]/m)**2
        
    return 1-t

def calculate_gini_index(Y_subsets):
    m = 0
    for i in Y_subsets:
        m += len(i)
    
    gin = 0 
    for i in Y_subsets:
        coef = len(i)/m
        gin += coef*gini(i)
        
    return gin

def split_data_set(data_X, data_Y, feature_index, threshold):
    left_X = []
    right_X = []
    left_Y = []
    right_Y = []
    for i in range(len(data_X)):
        if data_X[i][feature_index] < threshold:
            left_X.append(data_X[i])
            left_Y.append(data_Y[i])
        else:
            right_X.append(data_X[i])
            right_Y.append(data_Y[i])
    
    return left_X, left_Y, right_X, right_Y

def get_best_split(X, Y):
    X = np.array(X)
    best_gini_index = 99999
    best_feature = 0
    best_threshold = 0
    for i in range(len(X[0])):
        thresholds = sorted(set(X[:, i]))
        for t in thresholds:
            left_X, left_Y, right_X, right_Y = split_data_set(X, Y, i, t)
            if len(left_X) == 0 or len(right_X) == 0:
                continue
            gini_index = calculate_gini_index([left_Y, right_Y])
            if gini_index < best_gini_index:
                best_gini_index, best_feature, best_threshold = gini_index, i, t
                
    return best_feature, best_threshold

class Node:
    def __init__(self, predicted_class, depth):
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.depth = depth
        self.left = None
        self.right = None

def construct_tree(X, Y, max_depth, min_size, depth):
    Y2 = np.array(Y).reshape(len(Y),)
    classes = list(set(Y2))
    predicted_class = classes[np.argmax([np.sum(Y2 == c) for c in classes])]
    node = Node(predicted_class, depth)

    #check is pure
    if len(set(Y)) == 1:
        return node
    
    #check max depth reached
    if depth >= max_depth:
        return node

    #check min subset at node
    if len(Y) <= min_size:
        return node

    feature_index, threshold = get_best_split(X, Y)

    if feature_index is None or threshold is None:
        return node

    node.feature_index = feature_index
    node.threshold = threshold
    
    left_X, left_Y, right_X, right_Y = split_data_set(X, Y, feature_index, threshold)

    node.left = construct_tree(np.array(left_X), np.array(left_Y), max_depth, min_size, depth + 1)
    node.right = construct_tree(np.array(right_X), np.array(right_Y), max_depth, min_size, depth + 1)
    
    return node

def print_tree(node):
    if node.left is not None and node.right is not None:
        print("X" + str(node.feature_index) + " " + str(node.threshold))
        
    if node.left is not None:
        print_tree(node.left)
        
    if node.right is not None:
        print_tree(node.right)
        
def predict_class(root, X):
    node = root #root is a tree constructed using construct_tree method
    while node.left:
        if X[node.feature_index] < node.threshold:
            node = node.left
        else:
            node = node.right
            
    return node.predicted_class


In [9]:
m = 100
n = math.ceil(len(X_train.columns)**0.5)

random_features = []

for i in range(m):
    arr = []
    while True:
        if len(arr)>=n:
            break
            
        k = random.randint(0,59)
        if k not in arr:
            arr.append(k)
            
    random_features.append(arr)
    
random_features = np.array(random_features)

In [10]:
def get_bootstrap_samples(X):
    bootstraps = []
    columns = X.columns
    for i in random_features:
        arr = X[columns[i]]
        bootstraps.append(arr)
        
    return bootstraps
    

bootstraps = get_bootstrap_samples(X_train)

In [11]:
models = []

for i in bootstraps:
    model = construct_tree(i, Y_train, 50, 2, 0)
    print_tree(model)
    models.append(model)

In [12]:
final = []
for sample in X_test.values:
    predictions = []
    for i in models:
        pred = predict_class(i, sample)
        predictions.append(pred)
        
    final.append(np.argmax(predictions))
    

In [13]:
accuracy_score(Y_test, final)

0.46875

In [14]:
def gini(S):
    c = Counter(S)
    m = len(S)
    t = 0
    for i in c:
        t += (c[i]/m)**2
        
    return 1-t

def calculate_gini_index(Y_subsets):
    m = 0
    for i in Y_subsets:
        m += len(i)
    
    gin = 0 
    for i in Y_subsets:
        coef = len(i)/m
        gin += coef*gini(i)
        
    return gin


def draw_bootstrap(X_train, y_train):
    bootstrap_indices = list(np.random.choice(range(len(X_train)), len(X_train), replace = True))
    oob_indices = [i for i in range(len(X_train)) if i not in bootstrap_indices]
    
    X_bootstrap = X_train.iloc[bootstrap_indices].values
    y_bootstrap = y_train.iloc[bootstrap_indices].values
    X_oob = X_train.iloc[oob_indices].values
    y_oob = y_train.iloc[oob_indices].values
    
    return X_bootstrap, y_bootstrap, X_oob, y_oob


def oob_score(tree, X_test, y_test):
    mis_label = 0
    for i in range(len(X_test)):
        pred = predict_tree(tree, X_test[i])
        if pred != y_test[i]:
            mis_label += 1
            
    return mis_label / len(X_test)


def find_split_point(X_bootstrap, y_bootstrap, max_features):
    feature_ls = list()
    num_features = len(X_bootstrap[0])

    while len(feature_ls) <= max_features:
        feature_idx = random.sample(range(num_features), 1)
        if feature_idx not in feature_ls:
            feature_ls.extend(feature_idx)

    best_gini = 10**10
    node = None
    
    for feature_idx in feature_ls:
        for split_point in X_bootstrap[:,feature_idx]:
            left_child = {'X_bootstrap': [], 'y_bootstrap': []}
            right_child = {'X_bootstrap': [], 'y_bootstrap': []}

            # split children for continuous variables
            if type(split_point) in [int, float]:
                for i, value in enumerate(X_bootstrap[:,feature_idx]):
                    if value <= split_point:
                        left_child['X_bootstrap'].append(X_bootstrap[i])
                        left_child['y_bootstrap'].append(y_bootstrap[i])
                    else:
                        right_child['X_bootstrap'].append(X_bootstrap[i])
                        right_child['y_bootstrap'].append(y_bootstrap[i])
                        
            # split children for categoric variables
            else:
                for i, value in enumerate(X_bootstrap[:,feature_idx]):
                    if value == split_point:
                        left_child['X_bootstrap'].append(X_bootstrap[i])
                        left_child['y_bootstrap'].append(y_bootstrap[i])
                    else:
                        right_child['X_bootstrap'].append(X_bootstrap[i])
                        right_child['y_bootstrap'].append(y_bootstrap[i])
            
            subsets = left_child['y_bootstrap'] + right_child['y_bootstrap']
            gini_value = calculate_gini_index(subsets)
            
            if gini_value < best_gini:
                best_gini = gini_value
                left_child['X_bootstrap'] = np.array(left_child['X_bootstrap'])
                right_child['X_bootstrap'] = np.array(right_child['X_bootstrap'])
                
                node = {'information_gain': gini_value,
                        'left_child': left_child,
                        'right_child': right_child,
                        'split_point': split_point,
                        'feature_idx': feature_idx}

    return node


def terminal_node(node):
    y_bootstrap = node['y_bootstrap']
    pred = max(y_bootstrap, key = y_bootstrap.count)
    
    return pred


def split_node(node, max_features, min_samples_split, max_depth, depth):
    left_child = node['left_child']
    right_child = node['right_child']

    del(node['left_child'])
    del(node['right_child'])

    if len(left_child['y_bootstrap']) == 0 or len(right_child['y_bootstrap']) == 0:
        empty_child = {'y_bootstrap': left_child['y_bootstrap'] + right_child['y_bootstrap']}
        node['left_split'] = terminal_node(empty_child)
        node['right_split'] = terminal_node(empty_child)
        return

    if depth >= max_depth:
        node['left_split'] = terminal_node(left_child)
        node['right_split'] = terminal_node(right_child)
        return node

    if len(left_child['X_bootstrap']) <= min_samples_split:
        node['left_split'] = node['right_split'] = terminal_node(left_child)
        
    else:
        node['left_split'] = find_split_point(left_child['X_bootstrap'], left_child['y_bootstrap'], max_features)
        split_node(node['left_split'], max_depth, min_samples_split, max_depth, depth + 1)
        
    if len(right_child['X_bootstrap']) <= min_samples_split:
        node['right_split'] = node['left_split'] = terminal_node(right_child)
        
    else:
        node['right_split'] = find_split_point(right_child['X_bootstrap'], right_child['y_bootstrap'], max_features)
        split_node(node['right_split'], max_features, min_samples_split, max_depth, depth + 1)

In [15]:
def build_tree(X_bootstrap, y_bootstrap, max_depth, min_samples_split, max_features):
    root_node = find_split_point(X_bootstrap, y_bootstrap, max_features)
    split_node(root_node, max_features, min_samples_split, max_depth, 1)
    
    return root_node


def random_forest(X_train, y_train, n_estimators, max_features, max_depth, min_samples_split):
    tree_ls = list()
    oob_ls = list()
    for i in range(n_estimators):
        X_bootstrap, y_bootstrap, X_oob, y_oob = draw_bootstrap(X_train, y_train)
        tree = build_tree(X_bootstrap, y_bootstrap, max_features, max_depth, min_samples_split)
        tree_ls.append(tree)
        oob_error = oob_score(tree, X_oob, y_oob)
        oob_ls.append(oob_error)
        
    print("OOB estimate: {:.2f}".format(np.mean(oob_ls)))
    
    return tree_ls


def predict_tree(tree, X_test):
    feature_idx = tree['feature_idx']

    if X_test[feature_idx] <= tree['split_point']:
        if type(tree['left_split']) == dict:
            return predict_tree(tree['left_split'], X_test)
        else:
            value = tree['left_split']
            return value
        
    else:
        if type(tree['right_split']) == dict:
            return predict_tree(tree['right_split'], X_test)
        
        else:
            return tree['right_split']
        
        
def predict_rf(tree_ls, X_test):
    pred_ls = list()
    for i in range(len(X_test)):
        ensemble_preds = [predict_tree(tree, X_test.values[i]) for tree in tree_ls]
        final_pred = max(ensemble_preds, key = ensemble_preds.count)
        pred_ls.append(final_pred)
        
    return np.array(pred_ls)

In [16]:
n_estimators = 50
max_features = 8
max_depth = 10
min_samples_split = 2

model = random_forest(X_train, Y_train, n_estimators, max_features, max_depth, min_samples_split)

OOB estimate: 0.49


In [17]:
preds = predict_rf(model, trainX)

accuracy_score(trainY, preds)

0.532051282051282

In [24]:
model2 = RandomForestClassifier(criterion="gini", max_depth=8, min_samples_split=2,
                               oob_score=True, n_jobs=-1)

Y_trai = Y_train.values.reshape(len(Y_train),)
model2.fit(X_train, Y_trai)

RandomForestClassifier(max_depth=8, n_jobs=-1, oob_score=True)

In [25]:
preds = model2.predict(X_test)

In [26]:
accuracy_score(Y_test, preds)

0.90625

In [27]:
model2.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 8,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'oob_score': True,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [28]:
model3 = RandomForestClassifier(bootstrap=False)

Y_trai = Y_train.values.reshape(len(Y_train),)
model3.fit(X_train, Y_trai)

RandomForestClassifier(bootstrap=False)

In [29]:
preds = model3.predict(X_test)
accuracy_score(Y_test, preds)

0.75

In [31]:
pickle.dump(model3, open("MODEL_FILE.sav", "wb"))

In [32]:
import pickle
k = pickle.load(open("MODEL_FILE.sav","rb"))

In [36]:
test_X = np.genfromtxt("./train_X_rf.csv", delimiter=',', dtype=np.float64, skip_header=1)
test_Y = np.genfromtxt("./train_Y_rf.csv", delimiter=",", dtype=np.float64)
pred = k.predict(test_X)

In [37]:
accuracy_score(test_Y, pred)

0.9487179487179487

In [19]:
import os
os.listdir()

['.ipynb_checkpoints',
 '.virtual_documents',
 'algorithm.py',
 'MODEL_FILE.sav',
 'predict.py',
 'README.md',
 'train_X_rf.csv',
 'train_Y_rf.csv',
 'Untitled.ipynb',
 'validate.py',
 '__pycache__']